In [7]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV #网格搜索
# catboost安装方式：命令行输入 pip install -i https://pypi.tuna.tsinghua.edu.cn/simple catboost

In [8]:
# 读取数据
train = pd.read_csv('data/first_round_training_data.csv')
test = pd.read_csv('data/first_round_testing_data.csv')
train.head(100)

,Parameter1,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9,Parameter10,...,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,Quality_label
0,0.001660,0.591013,147.608373,38.186345,0.000421,0.000612,2286.523413,0.035407,0.593081,1.010385,...,0.168761,1.098755,36.955992,8.454598,11.438066,177.243120,338.729256,2.021704,0.079526,Pass
1,1.601749,0.015052,0.035864,51.130326,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385,...,11.649033,0.066671,225.632949,0.481860,20597.447822,3.723330,15.376190,0.986973,4.634376,Fail
2,0.098039,69.233685,0.080920,0.112265,0.000909,0.001972,2286.523413,0.035407,0.593081,1.010385,...,0.078213,110.079689,2.208138,0.073525,236.079314,0.064196,0.576302,33.875790,1.813727,Fail
3,18.181860,0.047325,0.018061,1.098102,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385,...,0.380281,0.011491,0.654517,0.025872,176.948915,0.029777,0.246726,27.117165,0.081819,Fail
4,0.012085,0.008749,0.005509,524.327396,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385,...,1.555672,38.613386,0.260989,0.009380,194.798039,0.055053,0.014725,13.569707,18.138496,Fail
5,0.004062,14.556483,0.786945,0.010545,0.000525,0.001623,2286.523413,0.035407,0.593081,1.010385,...,8.998940,6.392712,16.107479,1.016071,86.064258,0.576380,123.057489,16.133884,0.598517,Good
6,0.438449,1.232559,2.882699,0.610757,1.600654,0.464037,0.600827,17.850021,0.051850,0.010192,...,0.034557,0.000971,1.021246,1.791292,0.377312,0.035493,0.146690,41.285376,5.985572,Good
7,48159.917401,0.002987,14.863813,0.063287,1.434060,0.314162,0.600827,17.850021,0.051850,0.010192,...,0.255094,0.090395,21.035514,29.608630,0.581454,0.615609,3.321156,0.013034,0.045039,Good
8,1.456601,52.381578,20.625283,366.074831,1.434060,0.258497,0.600827,17.850021,0.051850,0.010192,...,49.151635,43.315938,0.581297,0.753916,0.610076,0.029777,0.007033,3.397996,20.322889,Pass
9,0.000109,4.784654,31.916672,20.672388,1.284806,0.175007,0.600827,17.850021,0.051850,0.010192,...,0.020348,58.681014,8.387914,33.362518,0.108174,1.356619,1.081445,0.056060,0.413597,Good


In [9]:
# 特征列表
features = ["Parameter5","Parameter6","Parameter7","Parameter8","Parameter9","Parameter10"]

In [10]:
# 标签转化
def quality_encoder(x):
    return {'Excellent':0,'Good':1,'Pass':2,'Fail':3}[x]

train['label'] = train.Quality_label.apply(quality_encoder)
train['label_Excellent'] = 1*(train['label'] == 0)
train['label_Good'] = 1*(train['label'] == 1)
train['label_Pass'] = 1*(train['label'] == 2)
train['label_Fail'] = 1*(train['label'] == 3)

In [11]:
# 本地多分类准确率评估
local_train = train.iloc[:5000,:].copy()
local_valid = train.iloc[5000:,:].copy()

model = CatBoostClassifier(iterations=3000,depth=10,learning_rate=0.001,verbose=100,loss_function='MultiClass',random_state=666)
model.fit(local_train.loc[:,features],local_train.label)
local_valid['prediction'] = model.predict(local_valid.loc[:,features])
print((local_valid.label == local_valid.prediction).mean())

0:	learn: 1.3834872	total: 479ms	remaining: 23m 55s
100:	learn: 1.2238462	total: 36.7s	remaining: 17m 34s
200:	learn: 1.1525707	total: 1m 14s	remaining: 17m 14s
300:	learn: 1.1161634	total: 1m 49s	remaining: 16m 20s
400:	learn: 1.0952108	total: 2m 22s	remaining: 15m 21s
500:	learn: 1.0818007	total: 2m 55s	remaining: 14m 36s
600:	learn: 1.0727771	total: 3m 25s	remaining: 13m 42s
700:	learn: 1.0654231	total: 3m 58s	remaining: 13m 2s
800:	learn: 1.0591525	total: 4m 31s	remaining: 12m 26s
900:	learn: 1.0541789	total: 5m 2s	remaining: 11m 45s
1000:	learn: 1.0496770	total: 5m 32s	remaining: 11m 3s
1100:	learn: 1.0460681	total: 5m 59s	remaining: 10m 19s
1200:	learn: 1.0421566	total: 6m 27s	remaining: 9m 40s
1300:	learn: 1.0390004	total: 6m 55s	remaining: 9m 2s
1400:	learn: 1.0362677	total: 7m 23s	remaining: 8m 26s
1500:	learn: 1.0333962	total: 7m 51s	remaining: 7m 51s
1600:	learn: 1.0311122	total: 8m 19s	remaining: 7m 16s
1700:	learn: 1.0288506	total: 8m 49s	remaining: 6m 43s
1800:	learn: 1.0

In [12]:
local_valid['prediction'] = model.predict(local_valid.loc[:,features])
(local_valid.label == local_valid.prediction).mean()

0.49

In [ ]:
its = [1000,1500,2000,2500,3000]
lrs = [0.01,0.025,0.05,0.001,0.0025,0.005,0.0001]
score = []
for it in its:
    for lr in lrs:
        print(str(it) + '  ' + str(lr))
        model = CatBoostClassifier(iterations=it,depth=10,learning_rate=lr,verbose=100,loss_function='MultiClass',random_state=666)
        model.fit(local_train.loc[:,features],local_train.label)
        local_valid['prediction'] = model.predict(local_valid.loc[:,features])
        score.append((local_valid.label == local_valid.prediction).mean())
print(score)

1000  0.01
0:	learn: 1.3806935	total: 412ms	remaining: 6m 51s
100:	learn: 1.1521133	total: 35.8s	remaining: 5m 18s
200:	learn: 1.0953843	total: 1m 9s	remaining: 4m 35s
300:	learn: 1.0726406	total: 1m 42s	remaining: 3m 57s
400:	learn: 1.0592755	total: 2m 11s	remaining: 3m 16s
500:	learn: 1.0489660	total: 2m 42s	remaining: 2m 41s
600:	learn: 1.0415319	total: 3m 12s	remaining: 2m 7s
700:	learn: 1.0347825	total: 3m 41s	remaining: 1m 34s
800:	learn: 1.0295456	total: 4m 7s	remaining: 1m 1s
900:	learn: 1.0247657	total: 4m 36s	remaining: 30.3s
999:	learn: 1.0204452	total: 5m 1s	remaining: 0us
1000  0.025
0:	learn: 1.3723926	total: 395ms	remaining: 6m 34s
100:	learn: 1.0801565	total: 35.8s	remaining: 5m 18s
200:	learn: 1.0492093	total: 1m 5s	remaining: 4m 19s
300:	learn: 1.0318180	total: 1m 32s	remaining: 3m 33s
400:	learn: 1.0203364	total: 2m	remaining: 2m 59s
500:	learn: 1.0070596	total: 2m 31s	remaining: 2m 30s
600:	learn: 0.9938750	total: 3m 7s	remaining: 2m 4s
700:	learn: 0.9829547	total: 

1100:	learn: 1.0773406	total: 6m 4s	remaining: 2m 12s
1200:	learn: 1.0726134	total: 6m 36s	remaining: 1m 38s
1300:	learn: 1.0686167	total: 7m 8s	remaining: 1m 5s
1400:	learn: 1.0648426	total: 7m 41s	remaining: 32.6s
1499:	learn: 1.0618146	total: 8m 12s	remaining: 0us
1500  0.005
0:	learn: 1.3834872	total: 395ms	remaining: 9m 51s
100:	learn: 1.2238462	total: 34.5s	remaining: 7m 57s
200:	learn: 1.1525707	total: 1m 9s	remaining: 7m 29s
300:	learn: 1.1161634	total: 1m 42s	remaining: 6m 49s
400:	learn: 1.0952108	total: 2m 15s	remaining: 6m 10s
500:	learn: 1.0818007	total: 2m 48s	remaining: 5m 36s
600:	learn: 1.0727771	total: 3m 17s	remaining: 4m 55s
700:	learn: 1.0654231	total: 3m 48s	remaining: 4m 20s
800:	learn: 1.0591525	total: 4m 19s	remaining: 3m 46s
900:	learn: 1.0541789	total: 4m 47s	remaining: 3m 11s
1000:	learn: 1.0496770	total: 5m 15s	remaining: 2m 37s
1100:	learn: 1.0460681	total: 5m 42s	remaining: 2m 3s
1200:	learn: 1.0421566	total: 6m 9s	remaining: 1m 32s
1300:	learn: 1.0390004

In [6]:
# 线上提交
model = CatBoostClassifier(iterations=3000,depth=8,learning_rate=0.005,verbose=100,loss_function='MultiClass',random_state=666)

model.fit(train.loc[:,features],train.label)

test['prediction'] = model.predict(test.loc[:,features])
test['prob_Excellent'] = 0.0
test['prob_Good'] = 0.0
test['prob_Pass'] = 0.0
test['prob_Fail'] = 0.0
test.loc[:,['prob_Excellent','prob_Good','prob_Pass','prob_Fail']] = model.predict_proba(test.loc[:,features])

NameError: name 'features' is not defined

In [11]:
# 提交用
prediction = test.groupby(['Group'],as_index=False)['prob_Excellent','prob_Good','prob_Pass','prob_Fail'].mean()
prediction.columns = ['Group','Excellent ratio','Good ratio','Pass ratio','Fail ratio']
prediction.to_csv('result/baseline_v3.csv',index=False)